In [1]:
from tools import *

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
tripadvisor_api_key = os.getenv("TRIPADVISOR_API_KEY")
rapid_api_key = os.getenv("RAPID_API_KEY")


In [2]:
import json

# Filepath to the JSON file
file_path = "data/vietnam-provinces.json"

# Open and read the JSON file
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

provinces = [data[i]['NameEn'] for i in range(len(data))]

In [17]:
provinces[0]

'Ha Noi'

In [26]:
import requests

url = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/list"

querystring = {"query":'Vung Tau',"page":"2"}

headers = {
	"x-rapidapi-key": "f01c3f6968msh4a12cf8c0c8f3c2p1d9e36jsne6d66dae12ae",
	"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

In [27]:
response.raise_for_status()  # Raise an error for bad responses
data = response.json()
data

{'link': 'https://www.tripadvisor.com/Restaurants-g303946-oa30-Vung_Tau_Ba_Ria_Vung_Tau_Province.html',
 'total_pages': 8,
 'current_page': 2,
 'total_items_count': 230,
 'items_count': 31,
 'results': [{'id': 22957317,
   'name': "Dw's Bar & Grill",
   'link': 'https://www.tripadvisor.com/Restaurant_Review-g303946-d22957317-Reviews-Dw_s_Bar_Grill-Vung_Tau_Ba_Ria_Vung_Tau_Province.html',
   'reviews': 54,
   'rating': 4.8,
   'price_range_usd': '$$ - $$$',
   'is_sponsored': True,
   'menu_link': 'https://www.dws.kitchen/',
   'reservation_link': None,
   'featured_image': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/29/f8/74/proudly-vietnamese-but.jpg',
   'latitude': 10.33163,
   'longitude': 107.08237,
   'has_delivery': False,
   'is_premium': True,
   'cuisines': ['International', 'Barbecue', 'Wine Bar']},
  {'id': 21350850,
   'name': 'Bun Hai San Be Be',
   'link': 'https://www.tripadvisor.com/Restaurant_Review-g303946-d21350850-Reviews-Bun_Hai_San_Be_Be-Vung_Tau_

In [ ]:
data['results'][0]

{'id': 15697908,
 'name': 'Mr.Hung Bar Restaurant',
 'link': 'https://www.tripadvisor.com/Restaurant_Review-g1544599-d15697908-Reviews-Mr_Hung_Bar_Restaurant-Ha_Giang_Ha_Giang_Province.html',
 'reviews': 275,
 'rating': 4.8,
 'price_range_usd': '$$ - $$$',
 'is_sponsored': False,
 'menu_link': None,
 'reservation_link': None,
 'featured_image': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/87/2e/8d/caption.jpg',
 'latitude': 22.805433,
 'longitude': 104.97557,
 'has_delivery': False,
 'is_premium': False,
 'cuisines': ['Italian',
  'Bar',
  'Pizza',
  'Asian',
  'Pub',
  'Vietnamese',
  'Israeli']}

In [20]:
import csv

# Assuming 'data' is already fetched as in your previous cells
results = data['results']

# Get the keys for CSV columns from the first result
if results:
    columns = results[0].keys()
    csv_filename = f"{provinces[1]}.csv"

    with open(csv_filename, "w", newline='', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=columns)
        writer.writeheader()
        writer.writerows(results)
    print(f"Saved {len(results)} rows to {csv_filename}")
else:
    print("No results found.")

Saved 30 rows to Ha Giang.csv
